### connect to mysql

In [ ]:
import pandas as pd
import pyodbc

def update_qry(conn):  #차량가액테이블 연식구분 A/B 설정
    cursor=conn.cursor()
    q=f"""
update  ins_cramt_info set nwcr_flgcd= case when mfg_ap_str_dthms>='20200701' then 'A' else 'B' end 
where opedt='20210101'
 and yytp='2020'
      """
    
    q=f"""
delete from ins_cramt_info
where opedt='20210101'
 and yytp='2020'
 and cnmcd='11R02'
 and nwcr_flgcd in ('A','B')
  """
    cursor.execute(q)
    return 'succ'


def sample_qry(conn):
    cursor=conn.cursor()
    q=f"""
select  * from ins_crdif_inq_agr_adm
-- where opedt='20210101'
-- and yytp='2020'
limit 1
      """
    cursor.execute(q)
    res=[]
    for row in cursor:
        res.append(row)
#         print(row)
#     conn.close()
    return res

#server: 개발 dev-core.. qa: qa_core.. 

def get_conn(db):
    if db=='dev':
        SERVER= 'dev-core-myaurora.cluster-cuwugfq70vdv.ap-northeast-2.rds.amazonaws.com'
        PWD=''
        PORT='3306'
    elif db=='qa':    
        SERVER= 'qa-core-myaurora.cluster-cuwugfq70vdv.ap-northeast-2.rds.amazonaws.com'
        PWD=''
        PORT='3306'
    else:
        SERVER= 'prod-core-myaurora.cluster-cuwugfq70vdv.ap-northeast-2.rds.amazonaws.com'
        PWD=''
        PORT='34242'
        
    drv = 'pymy'
    if drv == 'pyodbc':    
        connection_string = f"""
            DRIVER=MySQL ODBC 8.0 ANSI Driver;
            SERVER={SERVER};
            PORT={PORT};
            DATABASE=kismdbs;
            UID=kisdev;
            PWD={PWD};
            cursorclass=pyodbc.cursors.DictCursor;
        #     'charset=utf8;'
            MULTI_HOST=1;
        """    
        conn = pyodbc.connect(connection_string)
    else
        connection_string = f"""
            host = {SERVER},
            user = kisdev,
            password = {PWD},
            database = kismdbs,
            charset =utf8,
            port = 3306,
            cursorclass=pymysql.cursors.DictCursor
         """
         conn = pymysql.connect(connection_string)
    return conn

# insert sql 생성 ----------------
def make_sql(tbl,row):
    #값이 있는 컬럼의 index to list
    non_empty=[i for i in range(0,len(row)) if row[i] !=""]
    #값이 있는 컬럼명만 추출 to list
    cols=[str(x) for x in row.index[non_empty]]
    #값만 추출 to list
    params=[str(row[i]) for i in non_empty]
#     print(len(cols),len(params))
    q=f'''
      insert into {tbl}
      ( {",".join([str(i) for i in cols])} )
      values 
      ( {",".join(["?" for i in params])} )
     '''
    q= q.replace('\n','')
#     print(q)
    return q,params

In [ ]:
# sample qry 또는 update qry 실행
conn=get_conn('prod')
# res=update_qry(conn)
# print(res[0])
# conn.close()

res=sample_qry(conn)
print(res)
print('completed..')

### 시스템간 table 데이터 복사 ( row by row ) 

In [ ]:
conn_fr=get_conn('dev')
conn_to=get_conn('prod')
cursor_to=conn_to.cursor()

In [ ]:
# sources={"ins_cramt_info" : "opedt='20210101' and yytp='2020' limit 2"}
sources={"ins_cramt_info" : " yr='2020' and qtr='4' "}
 
for tbl,where in sources.items():
    # select from src
    q=f""" select  * from {tbl} where {where}  """
#     q=f""" select  * from {tbl}   """
    df=pd.read_sql(q,con=conn_fr)
    
    # insert to target
    cursor = conn_to.cursor()
    for i, row in df.iterrows():
        q,params = make_sql(tbl,row)
        cursor.execute(q, params)
        if i%1000 == 0:
            conn_to.commit()
            print(i,'번째')
        #     print(q,params)
    conn_to.commit()
    conn_to.close()
print('completed...')

### 시스템간 테이블 복사 (bulk)

In [ ]:
conn_fr = get_conn('dev')
conn_to = get_conn('prod')
cursor_to = conn_to.cursor()

In [ ]:
''' execute many 다건처리 '''

commit_cnt = 1000   #commit 단위
sources={"ins_cramt_info" : " yr='2020' and qtr='3' "}
 
for tbl,where in sources.items():
    # select from src
    q = f""" select  * from {tbl} where {where}  """
    df = pd.read_sql(q,con=conn_fr)
    df = df.fillna('')
    
    # insert to target
    cursor = conn_to.cursor()
    cursor.fast_executemany = True
    cls = ",".join([i for i in df.columns])
    cls_p = ",".join(["?" for i in df.columns])
    q_str= f''' insert into {tbl} ({cls}) values   ({cls_p}) '''
    cnt = 0
    while True:
        params = (tuple(row) for i,row in df.iloc[cnt:cnt+commit_cnt].iterrows() )
        cursor.executemany(q_str,params)
        conn_to.commit()
        cnt += commit_cnt
        print('commit:', cnt)
        if cnt >= len(df):
            break
            
conn_to.close()
print('completed...')

In [ ]:
# 임시사용 로직
import pandas as pd
df = pd.DataFrame([ str(i).split(',') for i in res])
df.columns = ['cd','nm','ss']
df['cd'] = df['cd'].map(lambda x: x.replace('(','').replace(')','')).replace("'",'')
df['nm'] = df['nm'].map(lambda x: x.replace('(','').replace(')','')).replace("'",'')
df.to_excel('c:/kpayins/temp/carnm.xlsx')

### excel to mysql db

In [ ]:
import pandas as pd
tbl='ins_crdif_inq_agr_adm'
sht='data'
path='c:/kpayins/temp/'+tbl+'_등록.xlsx'

tbl = 'ins_prs_cr_ctp'
sht = 'ins_prs_cr_ctp'
path = 'c:/kpayins/temp/carnm.xlsx'

# df=pd.read_excel('c:/kpayins/temp/ins_crdif_inq_agr_adm.xlsx',sheet_name='data',header=0,dtype='object')
df = pd.read_excel(path,sheet_name=sht,header=0,dtype='object',na_filter=False)

In [ ]:
conn = get_conn('prod')
cursor = conn.cursor()

for i,row in df.iterrows():
    q,params = make_sql(tbl,row)
    cursor.execute(q, params)
    if i%1000 == 0:
        conn.commit()
        print(i,'번째')
     #    print(q,params)
    conn.commit()
    conn.close()
print('completed...')

## 이하는 참고용

### sqlalchemy 활용- orm모델
https://docs.sqlalchemy.org/en/13/orm/tutorial.html

In [1]:
''' create engine with pymysql'''
import sqlalchemy as sa
import pandas as pd
from  sqlalchemy import MetaData

SERVER ='localhost:3306'
ID ='glad'
PWD ='glad9'
DB ='glad'

connection_string = 'mysql+pymysql://{}:{}@{}/{}'.format(ID, PWD, SERVER, DB)
engine = sa.create_engine(connection_string, echo=False, encoding='utf-8')

In [ ]:
# to pandas 
import pandas as pd
indata = pd.read_sql_query("select * from cars", engine)

In [ ]:
engine.table_names()

In [ ]:
# print each row
with engine.connect() as con:
    rs = con.execute('select * from roles')
    for row in rs:
        print(row)

In [ ]:
# fectch
rs= engine.execute('select * from cars')
rows=rs.fetchall()
print(rows)

rows=rs.first()
print(rows)

In [ ]:
# connect ->df 
with engine.connect() as con:
    rs = con.execute("SELECT * FROM cars")
#     df = pd.DataFrame(rs.fetchmany(size=5))
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
df

In [ ]:
#column속성 찾기
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_columns('roles')

In [ ]:
connection = engine.connect()
metadata = sa.MetaData()
table = sa.Table('cars', metadata, autoload=True, autoload_with=engine)
print(table.columns.keys())

# select * from table 과 같음
query = sa.select([table])

# 이때 query의 내용을 출력해보면 sql query인 것을 알 수 있음
# print(query)

# 쿼리 실행
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()

# 결과 print 이때 10개만 출력하도록 함. 단순한 set 자료구조의 형태를 하고 있음.
print(result_set[:10])

### pymysql raw driver
https://pymysql.readthedocs.io/en/latest/modules/cursors.html

In [2]:
import pymysql as my
import time
def get_conn(uid, upw):
    conn = my.connect(
        host = 'localhost',
        user = uid,
        password = upw,
        database = 'glad',
        charset = 'utf8',
        port = 3306,
#         cursorclass = my.cursors.DictCursor
    )
    return conn

conn = get_conn('glad','glad9')
try:
    with conn.cursor() as cursor:
        query = """
                select count(*) from roles
            """
        cursor.execute(query)
#         result = cursor.fetchone()
        result = cursor.fetchall()
           
finally:
    conn.close()
print(result)

((0,),)


###  bulk insert 속도비교 
1.sqlalchemy  
2.df.to_sql  
3.mysql.bulk insert  

In [9]:
''' delete all '''
conn = get_conn('glad','glad9')
cursor = conn.cursor()
cursor.execute('delete from roles')
conn.commit()
conn.close()

In [6]:
''' bulk insert 1: '''
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String,  create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
dicts = [{"id": i,"name":'name'+str(i),"desc": 'desc'+str(i)} for i in range(0,100000)]

Base = declarative_base()
class Roles(Base):
    __tablename__ = "roles"
    id = Column(Integer, primary_key=True)
    name = Column(String(60))
    description = Column(String(200))

t0 = time.time()
engine.execute(
        Roles.__table__.insert(),dicts)
print("SQLAlchemy Core: Total time for " + str(len(dicts)) +
        " records " + str(time.time() - t0) + " secs")

SQLAlchemy Core: Total time for 100000 records 6.440404415130615 secs


In [8]:
''' # bulk insert 2:   '''
from sqlalchemy import event,inspect
import pandas as pd
inspector = inspect(engine)

ls = [[i,'name'+str(i),'desc'+str(i)] for i in range(0,100000)]

df = pd.DataFrame(ls)
df.columns = [ i['name'] for i in inspector.get_columns('roles')]  # get column names

# conn = get_conn('glad','glad9')
# cursor = conn.cursor()
@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(
    conn, cursor, statement, params, context, executemany
    ):
#     print('ss',executemany)  
    if executemany:
        cursor.fast_executemany = True

# event.listen(engine, "before_cursor_execute", receive_before_cursor_execute)
t0 = time.time()
df.to_sql('roles', engine, index=False, if_exists="append",schema='glad')
print("Total time for " + str(len(ls)) + " records " + str(time.time() - t0) + " sec")

Total time for 100000 records 8.088884830474854 sec


In [10]:
''' bulk insert 3: '''
import time
conn = get_conn('glad','glad9')
cursor = conn.cursor()
q_str= f''' insert into roles (id,name,description) values (%s,%s,%s) '''
ls = [[i,'name'+str(i),'desc'+str(i)] for i in range(0,100000)]
# row  = tuple(tuple(x) for x in ls)
row = tuple(ls)

t0 = time.time() 
cursor.executemany(q_str,row)

conn.commit()
print("core: Total time for " + str(len(row)) +
        " records " + str(time.time() - t0) + " sec")

core: Total time for 100000 records 8.784225225448608 sec
